<a href="https://colab.research.google.com/github/Mattlee10/CMD-HAR/blob/main/CMD_HAR_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Shared Encoder
class SharedEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SharedEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.fc(x)

# Private Encoder
class PrivateEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(PrivateEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.fc(x)

# Disentangler
class Disentangler(nn.Module):
    def __init__(self, shared_dim, private_dim, output_dim):
        super(Disentangler, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(shared_dim + private_dim, output_dim),
            nn.ReLU()
        )

    def forward(self, shared, private):
        combined = torch.cat((shared, private), dim=1)
        return self.fc(combined)

# Classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

# CMD-HAR 전체 구조
class CMDHARModel(nn.Module):
    def __init__(self, input_dim, shared_dim, private_dim, num_classes):
        super(CMDHARModel, self).__init__()
        self.shared_encoder = SharedEncoder(input_dim, shared_dim)
        self.private_encoder = PrivateEncoder(input_dim, private_dim)
        self.disentangler = Disentangler(shared_dim, private_dim, shared_dim + private_dim)
        self.classifier = Classifier(shared_dim + private_dim, num_classes)

    def forward(self, x):
        shared_feat = self.shared_encoder(x)
        private_feat = self.private_encoder(x)
        disentangled_feat = self.disentangler(shared_feat, private_feat)
        out = self.classifier(disentangled_feat)
        return out

# 예시 실행 (Colab에서 테스트 가능)
if __name__ == "__main__":
    input_dim = 50
    model = CMDHARModel(input_dim=input_dim, shared_dim=32, private_dim=32, num_classes=6)
    dummy_input = torch.randn(32, input_dim)
    output = model(dummy_input)
    print("Output shape:", output.shape)

Output shape: torch.Size([32, 6])


In [ ]:
# 손실 함수 & 옵티마이저 설정

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 데이터 로더 준비
# PAMAP2 데이터셋을 전처리해서 Dataset/DataLoader로 감싸기
# batch_size=32로 설정

In [ ]:
# 훈련 루프(Training Loop) 작성

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()


# 검증 & 전이 실험
# IMU만으로 학습한 뒤 PPG 데이터로 평가해보고
# (accuracy, F1 등) 지표를 기록

# 결과 시각화 및 기록
# Epoch별 손실/정확도 곡선 그리기
# 전이 성능 비교 표 정리

이 코드는 논문에서 제안한 Cross-Modal Disentaglement (CMD) 구조를 그대로 반영해서, 공통 특징(Private)을 분리->결합->분류 과정을 모듈화한 것

1. SharedEncoder

- 역할: 서로 다른 센서(IMU, PPG 등)에서 공통적으로 나타나는 패턴을 학습

- 구현: 입력 특징(input_dim)을 받아 은닉공간(hidden_dim)으로 토영 -> ReLU

- 이유: '사람이 걷는 신호'처럼 센서 종류에 관계없이 공유되는 행동 표현을 뽑아내기 위해

In [ ]:
class SharedEncoder(nn.Module):
  ...
  self.fc = nn.Sequential(
      nn.Linear(input_dim, hidden_dim),
      nn.ReLU()
  )

2. PrivateEncoder

- 역할: 각 센서만 갖고 있는 고유한 신호 특성을 학습

- 구현: SharedEncoder와 같은 구조지만, 학습되는 파라미터가 별개

- 이유: 'PPG의 맥파 형태나'나 'IMU의 가속도 진동' 같은 센서별 디테일을 따로 분리해서 표현하기 위해

In [ ]:
class PrivateEncoder(nn.Module):
  ...
  self.fc = nn.Sequential(
      nn.Linear(input_dim, hidden_dim),
      nn.ReLU()
  )

3. Disentangler

- 역할: Shared와 Private 피처를 결합(concatenate)한 뒤, 다시 '공통 <-> 고유' 특징을 효괒거으로 분리/재조합

- 구현: concat(shared, private) -> 은식 출력(output_dim) -> ReLU

- 이유: 단순 결합만 하면 정보가 뒤섞이기 때문에, 분리된 표현을 더 명확하게 재구성하기 위함

In [ ]:
class Disentagler(nn.Module):
  ...
  combined = torch.cat((shared, private), dim=1)
  return self.fc(combined)

4. Classifier

- 역할: Disentangler가 만든 통합 표현을 받아 최종 활동 레이블(걸음/달리기/기타) 예측

- 구현: 두 개의 선형 계측 + 중간 ReLU

- 이유: disentangled feature를 분류기에 투입해 HAR 과제를 해결

In [ ]:
class Classifier(nn.Module):
  ...
  self.fc = nn.Sequential(
      nn.Linear(input_dim, 64),
      nn.ReLU()
      nn.Linear(64, num_classes)
  )

5. CMDHARModel (전체 파이프라인)

- 순서:

1. SharedEncoder -> 공통 피처
2. PrivateEncoder -> 고유 피처
3. Disentangler -> 두 피처 합성/분리
4. Classifier -> 활동 예측

- 이유: 모듈화를 통해 각 단계(공통/고유/분리/분류)를 명확히 분리하고, 실험/디버깅/확장을 쉽게 하기 위함

In [ ]:
class CMDHARModel(nn.Module):
  def foward(self, x):
      shared_feat = self.shared_encoder(x)
      pricate_feat = self.private_encoder(x)
      disentangled_feat = self.disentangler(shared_feat, private_feat)
      out = self.classifier(disentangled_feat)
      return out

6. 더미 입력 테스트

- dummy_input = torch.randn(32,50)

- 목적: 배치 크기 32, 특성 차원 50인 랜덤 데이터를 넣어보면서 '모델 전체가 잘 연결됐는지', '출력 차원이 (32, num_classes)인지' 확인하기 위함

In [ ]:
example_output = model(dummy_input)
print("Output shape:", example_output.shape)

이 구조 덕분에

- 센서 종류가 바뀌어도 SharedEncoder가 공통 행동 신호를 잡아내고,

- PricateEncoder가 센서별 노이즈=신호를 분리해 내며,

- Disentangler가 두 표현을 최적의 형태로 결합/분리해 주기 때문에

- Zero-shot transfer처럼 'IMU로 학습 -> PPG에 바로 적용'이 가능해지는 것

- https://www.kaggle.com/code/avrahamcalev/time-series-models-pamap2-dataset

- https://archive.ics.uci.edu/dataset/231/pamap2+physical+activity+monitoring
